In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3) #input -? OUtput? RF
        self.bn1 = nn.BatchNorm2d(16)
        self.dp1 = nn.Dropout2d(0.2)

        self.conv2 = nn.Conv2d(16, 16, 3) #input -? OUtput? RF+
        self.bn2 = nn.BatchNorm2d(16)
        self.dp2 = nn.Dropout2d(0.2)
        
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(16, 8, 1)
        
        self.conv4 = nn.Conv2d(8, 16, 3) #input -? OUtput? RF
        self.bn4 = nn.BatchNorm2d(16)
        self.dp4 = nn.Dropout2d(0.2)

        self.conv5 = nn.Conv2d(16, 16, 3) #input -? OUtput? RF
        self.bn5 = nn.BatchNorm2d(16)
        self.dp5 = nn.Dropout2d(0.2)

        self.conv6 = nn.Conv2d(16, 16, 3) #input -? OUtput? RF
        self.bn6 = nn.BatchNorm2d(16)
        self.dp6 = nn.Dropout2d(0.2)

        self.conv7 = nn.Conv2d(16, 16, 3) #input -? OUtput? RF
        self.bn7 = nn.BatchNorm2d(16)
        self.dp7 = nn.Dropout2d(0.2)

        self.conv8 = nn.Conv2d(16, 10, 4) #input -? OUtput? RF
        

    def forward(self, x):
        x = self.dp1(self.bn1(F.relu(self.conv1(x))))
        x = self.pool1(self.dp2(self.bn2(F.relu(self.conv2(x)))))
        x = F.relu(self.conv3(x))
        x = self.dp4(self.bn4(F.relu(self.conv4(x))))
        x = self.dp5(self.bn5(F.relu(self.conv5(x))))
        x = self.dp6(self.bn6(F.relu(self.conv6(x))))
        x = self.dp7(self.bn7(F.relu(self.conv7(x))))
        x = self.conv8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
         Dropout2d-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
         Dropout2d-6           [-1, 16, 24, 24]               0
         MaxPool2d-7           [-1, 16, 12, 12]               0
            Conv2d-8            [-1, 8, 12, 12]             136
            Conv2d-9           [-1, 16, 10, 10]           1,168
      BatchNorm2d-10           [-1, 16, 10, 10]              32
        Dropout2d-11           [-1, 16, 10, 10]               0
           Conv2d-12             [-1, 16, 8, 8]           2,320
      BatchNorm2d-13             [-1, 16, 8, 8]              32
        Dropout2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss

In [9]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.5)
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    val_loss = test(model, device, test_loader)
    if(epoch==10 or epoch>=15):
      scheduler.step()
    for param_group in optimizer.param_groups:
      print(param_group['lr'],epoch)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0826, Accuracy: 9740/10000 (97%)

0.1 1


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0610, Accuracy: 9814/10000 (98%)

0.1 2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0227, Accuracy: 9919/10000 (99%)

0.00625 17


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0228, Accuracy: 9921/10000 (99%)

0.003125 18


loss=0.04980563744902611 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.30it/s]



Test set: Average loss: 0.0225, Accuracy: 9918/10000 (99%)

0.0015625 19
